In [10]:
wikipedia_tool_def = {
    "type": "function",
    "function": {
        "name": "wikipedia_search_tool",
        "description": "Searches for a Wikipedia article summary by query string.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search keywords for the Wikipedia article."
                },
                "sentences": {
                    "type": "integer",
                    "description": "Number of sentences in the summary.",
                    "default": 5
                }
            },
            "required": ["query"]
        }
    }
}
tools = [wikipedia_tool_def, wikipedia_tool_def]

tools_ = ""

for i, tool in enumerate(tools):
    name = f"tool name: {tool['function']['name']}; "
    description = f"tool goal: {tool['function']['description']}; "
    parameters = f"tool parameters: {tool['function']['parameters']['properties']}"
    
    tools_ += f"{i+1}. {name} {description} {parameters} \n"

print(tools_)

1. tool name: wikipedia_search_tool;  tool goal: Searches for a Wikipedia article summary by query string.;  tool parameters: {'query': {'type': 'string', 'description': 'Search keywords for the Wikipedia article.'}, 'sentences': {'type': 'integer', 'description': 'Number of sentences in the summary.', 'default': 5}} 
2. tool name: wikipedia_search_tool;  tool goal: Searches for a Wikipedia article summary by query string.;  tool parameters: {'query': {'type': 'string', 'description': 'Search keywords for the Wikipedia article.'}, 'sentences': {'type': 'integer', 'description': 'Number of sentences in the summary.', 'default': 5}} 



In [6]:
uv add torch

/home/padeiroc/ai_project/agents/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import gc

# --- Third-party ---
import torch
from dotenv import load_dotenv
from huggingface_hub import login
from utils.api_base_url import ApiConfig
from diffusers import StableDiffusionPipeline, AutoPipelineForText2Image, DiffusionPipeline

load_dotenv(override=True)
hf_token = os.getenv(ApiConfig.HUGGING_FACE_API_TOKEN)

ModuleNotFoundError: No module named 'diffusers'

In [ ]:

## Decision Tree: Which Pipeline to Use?
# ```
# Do you know the exact model family?
# ├─ YES, it's Stable Diffusion v1.x/v2.x
# │  └─> Use StableDiffusionPipeline (most explicit control)
# │
# ├─ YES, it's FLUX
# │  └─> Use FluxPipeline (FLUX-specific optimizations)
# │
# ├─ NO, could be anything
# │  └─> Use DiffusionPipeline (maximum flexibility)
# │
# └─ Want task-specific features for text-to-image?
#    └─> Use AutoPipelineForText2Image (clear intent + optimizations)

class ImageGenerator:
    
    def __init__(self, model_id:str = "stabilityai/stable-diffusion-xl-base-1.0",torch_dtype=torch.float16):
        login(hf_token, add_to_git_credential=True)
        self.torch_dtype = torch_dtype
        self.model_id = model_id
        
        self.pipeline = AutoPipelineForText2Image.from_pretrained(
                    self.model_id,
                    self.torch_dtype,
                    variant="fp16")
    
    def memory_optimizer(self, optimizer_type:str="offload"):
        if optimizer_type == "offload":
            self.pipeline.enable_model_cpu_offload()  # Aggressive memory saving
            
        elif optimizer_type == "vae_slicing":
            self.pipeline.enable_vae_slicing()         # Reduce memory for VAE
            
        else:
            self.pipeline.enable_attention_slicing()   # Reduce memory for attention
    
    def generate(self, prompt: str="", negative_prompt: str = "", size:dict = (512, 512),  pipeline_type:str ="text2Image", model_id:str = None, device:str="cuda"):
        
        if pipeline_type == "text2Image":
            
            image =  self.pipeline(
                prompt = prompt,
                negative_prompt = negative_prompt,
                num_inference_steps = 40,
                guidance_scale = 7.5,
                height = size[0],
                width = size[1]
                ).to(device)
            
            return image[0]
            
        if pipeline_type == "diffusion":
            
            if model_id is None:
                model_id = "runwayml/stable-diffusion-v1-5"
            # Explicitly for Stable Diffusion v1.x/v2.x
            self.pipeline = DiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                safety_checker=None  # Can disable safety checker
            )
            self.pipeline = self.pipeline.to("cuda")

            # Classic SD interface
            image = self.pipeline(
                prompt=prompt,
                negative_prompt = negative_prompt,
                num_inference_steps = 50,
                guidance_scale = 7.5,
                height = size[0],
                width = size[1]
            )
            return image[0]
    
        if pipeline_type == "stableDiffusion":
            
            if model_id is None:
                model_id = "runwayml/stable-diffusion-v1-5"
                
            # Explicitly for Stable Diffusion v1.x/v2.x
            self.pipeline = StableDiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                safety_checker=None  # Can disable safety checker
            )
            self.pipeline = self.pipeline.to("cuda")

            # Classic SD interface
            image = self.pipeline(
                prompt=prompt,
                negative_prompt = negative_prompt,
                num_inference_steps = 50,
                guidance_scale = 7.5,
                height = size[0],
                width = size[1]
            )
            return image[0]

In [1]:
from datetime import datetime

print(datetime.now().strftime("%Y-%m-%d-%h"))
print(datetime.now())

dt = datetime.now()

hour = dt.hour
minute = dt.minute
second = dt.second

name = f"image_{dt.strftime("%Y_%m_%d")}{dt.hour}{dt.minute}{dt.second}"

print(name)



2025-10-17-Oct
2025-10-17 22:37:18.739989
image_2025_10_17223718


In [1]:
import os
import pandas as pd
def load_and_prepare_data( csv_path: str=os.path.join(os.getcwd(),"dataset/coffee_sales.csv")) -> pd.DataFrame:
    """Load CSV and derive date parts commonly used in charts."""
    df = pd.read_csv(csv_path)
    
    # Be tolerant if 'date' exists
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df["quarter"] = df["date"].dt.quarter
        df["month"] = df["date"].dt.month
        df["year"] = df["date"].dt.year
    return df

In [ ]:
uv add --active matplotlib

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Assuming df is already loaded
df = load_and_prepare_data()

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')

plt.figure(figsize=(10,6))
plt.scatter('cash_type', 'price', data=df, hue='coffee_name', marker='', s=50, alpha=0.5, c='b', edgecolor='red')
plt.title('Coffee Price by Type and Name', fontsize=18)
plt.xlabel('Cash Type', fontsize=14)
plt.ylabel('Price', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)

plt.savefig('/home/padeiroc/ai_project/data_processing_agent/out_puts/generate_chart_2025_10_1817109.png', dpi=300)
plt.close()

ModuleNotFoundError: No module named 'matplotlib'